# SuperINVI — Clean Setup (Fixed)
This notebook is **auto-generated** from your original upload and fixes:
- Duplicated Node.js/npm install steps
- Commented-out `%%writefile` blocks (which prevented `server.js`/`index.html` from being created)
- Non-deterministic cell order
- Missing CORS between ports 8000 and 3000

**Run cells top-to-bottom.** Backend runs on **:3000**, frontend served on **:8000**.


In [1]:
# Kill stale ports (ignore errors if nothing is running)
!fuser -k 3000/tcp || true
!fuser -k 8000/tcp || true


In [2]:
# Install Node.js LTS (NodeSource) + serve
!apt-get update -y
!curl -fsSL https://deb.nodesource.com/setup_lts.x | bash -
!apt-get install -y nodejs
!npm install -g serve

# Verify
!node -v
!npm -v


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,008 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,798 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all 

In [3]:
# Create and enter project directory
!mkdir -p /content/superinvi
%cd /content/superinvi

# Init npm and install backend deps
!npm init -y
!npm install express body-parser cors

/content/superinvi
⠙Wrote to /content/superinvi/package.json:

{
  "name": "superinvi",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}



⠙⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 70 packages, and audited 71 packages in 3s
⠦
⠦16 packages are looking for funding
⠦  run `npm fund` for details
⠦
found 0 vulnerabilities
⠦

In [4]:
import os
from google.colab import userdata

# Load Colab secrets into environment so Node.js can access them
os.environ["Gemini_API"] = userdata.get("Gemini_API")
os.environ["invi_ngrok"] = userdata.get("invi_ngrok")

print("✅ Environment variables set: Gemini_API and invi_ngrok")


✅ Environment variables set: Gemini_API and invi_ngrok


In [5]:
%%writefile server.js
const express = require("express");
const bodyParser = require("body-parser");
const cors = require("cors");

const app = express();
app.use(bodyParser.json());
app.use(cors({ origin: true }));

// Gemini API key
const GEMINI_API = process.env.Gemini_API;

app.get("/", (req, res) => {
  res.send("✅ SuperINVI backend is running. Use POST /generate");
});

app.post("/generate", async (req, res) => {
  const { description = "" } = req.body || {};

  try {
    if (!GEMINI_API) throw new Error("Missing Gemini API key");

    const response = await fetch(
      `https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key=${GEMINI_API}`,
            {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({
          contents: [{
            role: "user",
            parts: [{ text: `Generate a responsive HTML and CSS template for: ${description}. Do not include markdown fences, only pure HTML and CSS code.` }]
          }]
        }),
      }
    );

    const data = await response.json();

    console.log("Gemini API raw response:", JSON.stringify(data, null, 2));

    let generatedHTML = data?.candidates?.[0]?.content?.parts?.[0]?.text?.trim();

    if (!generatedHTML) {
      // Include the Gemini API's error message if available
      const errorMessage = data?.error?.message || "Could not generate a site for your request.";
      return res.json({
        html: `<p>⚠️ ${errorMessage} Try rephrasing your description.</p>`
      });
    }

    res.json({
      html: generatedHTML,
      code: generatedHTML
       });

  } catch (err) {
    console.error("❌ Error:", err.message);
    res.json({
      html: `<p>⚠️ Gemini API Error: ${err.message}</p>`
    });
  }
});

const PORT = process.env.PORT || 3000;
app.listen(PORT, () => console.log(`✅ SuperINVI Gemini API running on port ${PORT}`));

Writing server.js


In [6]:
%%writefile index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>SUPER INVI</title>
  <meta name="description" content="Generate a portfolio, preview and iterate via chat." />
  <style>
    :root{
      --bg: linear-gradient(180deg,#071026 0%, #08142b 100%);
      --muted: #dbeafe;
      --accent: #60a5fa;
      --accent-2: #f59e0b;
      --card: rgba(255,255,255,0.03);
      --radius: 12px;
      --gap: 16px;
      --chat-w: 22%;
    }
    *{box-sizing:border-box}
    html,body{height:100%;margin:0;font-family:Inter,system-ui,-apple-system,"Segoe UI",Roboto,Arial}
    body{background:var(--bg);color:var(--muted);padding:18px;display:flex;justify-content:center;align-items:stretch;-webkit-font-smoothing:antialiased}
    .app{width:100%;max-width:1400px;height:calc(100vh - 36px);display:flex;position:relative;gap:var(--gap)}

    /* HERO (initial) */
    .hero{flex:1;border-radius:var(--radius);background:linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.01));display:flex;align-items:center;justify-content:center;padding:40px;transition:all .42s}
    .hero.hidden{opacity:0;transform:translateY(-8px) scale(.998);pointer-events:none;height:0;padding:0;margin:0}
    .hero-inner{width:100%;max-width:980px;text-align:center}
    .title{font-size:28px;font-weight:700;margin:0 0 8px;color:#fff}
    .subtitle{margin:0 0 18px;color:rgba(255,255,255,0.86)}
    .controls{display:flex;gap:10px;justify-content:center;align-items:center}
    .input{width:60%;min-width:260px;padding:12px 14px;border-radius:10px;border:1px solid rgba(255,255,255,0.04);background:rgba(255,255,255,0.02);color:var(--muted);outline:none}
    .btn{padding:10px 16px;border-radius:10px;border:0;cursor:pointer;font-weight:700;background:var(--accent-2);color:#071120}
    .btn.ghost{background:rgba(255,255,255,0.06);color:var(--muted);border:1px solid rgba(255,255,255,0.03)}

    /* SPLIT layout (20% chat, 80% preview) */
    .shell{display:flex;gap:var(--gap);width:100%;height:100%}
    .hidden{display:none !important}

    .chat{width:var(--chat-w);min-width:240px;max-width:360px;background:linear-gradient(180deg, rgba(255,255,255,0.015), rgba(255,255,255,0.01));border-radius:var(--radius);border:1px solid rgba(255,255,255,0.03);display:flex;flex-direction:column;overflow:hidden;box-shadow:0 8px 30px rgba(2,6,23,0.6)}
    .chat-header{display:flex;gap:12px;align-items:center;padding:14px;border-bottom:1px solid rgba(255,255,255,0.02)}
    .avatar{width:44px;height:44px;border-radius:10px;background:linear-gradient(135deg,var(--accent),#7c3aed);display:flex;align-items:center;justify-content:center;font-weight:800;color:#031129}
    .meta{display:flex;flex-direction:column}
    .meta .t{font-weight:700;color:#e6eef8}
    .meta .s{font-size:12px;color:rgba(219,234,254,0.85)}

    .chat-body{padding:12px;flex:1;overflow:auto;display:flex;flex-direction:column;gap:10px}
    .bubble{max-width:92%;padding:10px 12px;border-radius:12px;font-size:13px;line-height:1.35;box-shadow:0 6px 18px rgba(2,6,23,0.5);word-break:break-word}
    .bubble.user{align-self:flex-end;background:linear-gradient(90deg,var(--accent),#7dd3fc);color:#02283a;border-bottom-right-radius:6px}
    .bubble.assistant{align-self:flex-start;background:rgba(255,255,255,0.035);color:var(--muted);border-bottom-left-radius:6px}

    .chat-foot{padding:12px;border-top:1px solid rgba(255,255,255,0.02);display:flex;gap:8px;align-items:center;background:linear-gradient(180deg,transparent, rgba(0,0,0,0.02))}
    .chat-input{flex:1;min-height:44px;border-radius:10px;padding:10px;border:1px solid rgba(255,255,255,0.04);background:rgba(255,255,255,0.02);color:var(--muted);outline:none;resize:none}

    .preview{flex:1;display:flex;flex-direction:column;gap:12px}
    .preview-controls{display:flex;justify-content:space-between;align-items:center;gap:12px}
    .controls-left{display:flex;gap:12px;align-items:center;flex:1}
    .panel{background:var(--card);border-radius:var(--radius);border:1px solid rgba(255,255,255,0.03);height:calc(100% - 48px);overflow:hidden;display:flex;flex-direction:column}
    .preview-area{display:flex;gap:12px;padding:12px;height:100%}
    .panel-inner{flex:1;border-radius:10px;overflow:hidden;background:white;position:relative}
    iframe#preview{width:100%;height:100%;border:0;display:block}
    pre#code{margin:0;padding:14px;height:100%;overflow:auto;background:#0e1116;color:#d7e6ff;font-family:ui-monospace,Menlo,Monaco,"Roboto Mono",monospace;font-size:13px;line-height:1.45;white-space:pre-wrap}

    @media(max-width:1000px){ .chat{display:none} .input{width:100%} }
  </style>
</head>
<body>
  <div class="app" id="app">
    <!-- HERO (initial) -->
    <div class="hero" id="hero">
      <div class="hero-inner">
        <div class="title"> SUPER INVI </div>
        <div class="subtitle">Describe the website (hero, projects, contact). Click Generate to create and iterate via chat.</div>
        <div class="controls">
          <input id="heroInput" class="input" placeholder="e.g. Freelancer portfolio with hero, projects, testimonials, contact form" />
          <button id="heroGen" class="btn">Generate</button>
          <button id="heroSample" class="btn ghost">Sample</button>
        </div>
      </div>
    </div>

    <!-- SPLIT (hidden at start) -->
    <div class="shell hidden" id="shell" aria-hidden="true">
      <aside class="chat" id="chat">
        <div class="chat-header">
          <div class="avatar">IN</div>
          <div class="meta"><div class="t">INVI Generator</div><div class="s">Iterative conversation for edits</div></div>
        </div>

        <div class="chat-body" id="chatBody" role="log" aria-live="polite"></div>

        <div class="chat-foot">
          <textarea id="chatInput" class="chat-input" placeholder="Ask for edits (e.g. 'Add navbar and testimonials')"></textarea>
          <button id="chatSend" class="btn">Send</button>
          <button id="chatClear" class="btn ghost">Clear</button>
        </div>
      </aside>

      <section class="preview" id="previewCol">
        <div class="preview-controls">
          <div class="controls-left">
            <!-- NOTE: NO generate/search field here per your request -->
            <div style="color:rgba(255,255,255,0.75);font-size:13px">Preview</div>
            <button id="download" class="btn ghost hidden">Download</button>
          </div>
          <div style="display:flex;gap:8px;align-items:center">
            <button id="toggle" class="btn ghost hidden">View Code</button>
            <button id="copy" class="btn ghost hidden">Copy</button>
          </div>
        </div>

        <div class="panel">
          <div class="preview-area">
            <div class="panel-inner" id="previewPanel">
              <!-- sandboxed iframe: scripts and forms allowed but top-level navigation blocked -->
              <iframe id="preview" title="Generated preview" sandbox="allow-scripts allow-forms"></iframe>
            </div>

            <div class="panel-inner hidden" id="codePanel">
              <pre id="code"></pre>
            </div>
          </div>
        </div>
      </section>
    </div>
  </div>

  <script>
  (function(){
    // ===== CONFIGURE: replace with your backend URL (no trailing slash) =====
    const BASE_URL = "{{NGROK_URL}}";
    // ======================================================================

    // DOM
    const hero = document.getElementById('hero');
    const heroInput = document.getElementById('heroInput');
    const heroGen = document.getElementById('heroGen');
    const heroSample = document.getElementById('heroSample');

    const shell = document.getElementById('shell');
    const chatBody = document.getElementById('chatBody');
    const chatInput = document.getElementById('chatInput');
    const chatSend = document.getElementById('chatSend');
    const chatClear = document.getElementById('chatClear');

    const download = document.getElementById('download');
    const toggle = document.getElementById('toggle');
    const copyBtn = document.getElementById('copy');

    const previewIframe = document.getElementById('preview');
    const previewPanel = document.getElementById('previewPanel');
    const codePanel = document.getElementById('codePanel');
    const codeEl = document.getElementById('code');

    // state
    let lastGenerated = { html:'', code:'' };
    let showingCode = false;
    let isGenerating = false;

    // append message bubble
    function appendBubble(role, text){
      const d = document.createElement('div');
      d.className = 'bubble ' + (role === 'user' ? 'user' : 'assistant');
      d.textContent = text;
      chatBody.appendChild(d);
      chatBody.scrollTop = chatBody.scrollHeight;
      return d;
    }

    function showSplit(){
      hero.classList.add('hidden');
      hero.setAttribute('aria-hidden','true');
      shell.classList.remove('hidden');
      shell.setAttribute('aria-hidden','false');
      // focus chat input for quick edits
      setTimeout(()=> chatInput.focus(), 200);
    }

    function setControlsVisible(v){
      if (v){ toggle.classList.remove('hidden'); copyBtn.classList.remove('hidden'); download.classList.remove('hidden'); }
      else { toggle.classList.add('hidden'); copyBtn.classList.add('hidden'); download.classList.add('hidden'); }
    }

    function setShowingCode(flag){
      showingCode = !!flag;
      if (showingCode){
        previewPanel.classList.add('hidden');
        codePanel.classList.remove('hidden');
        toggle.textContent = 'View Preview';
      } else {
        codePanel.classList.add('hidden');
        previewPanel.classList.remove('hidden');
        toggle.textContent = 'View Code';
      }
    }

    function persist(){ try { sessionStorage.setItem('invi_last', JSON.stringify(lastGenerated)); } catch(e){} }
    function restore(){
      try {
        const raw = sessionStorage.getItem('invi_last');
        if (!raw) return false;
        const parsed = JSON.parse(raw);
        if (!parsed) return false;
        lastGenerated = parsed;
        // put safe (base-target) html into iframe
        previewIframe.srcdoc = ensureBaseTarget(parsed.html || '');
        codeEl.textContent = parsed.code || parsed.html || '';
        setControlsVisible(true);
        setShowingCode(false);
        hero.classList.add('hidden');
        shell.classList.remove('hidden');
        return true;
      } catch(e){ return false; }
    }

    // build safe HTML for the iframe: insert <base target="_blank"> so links open new tabs
    function ensureBaseTarget(html){
      if (!html) return html;
      try {
        // if there's a <head> tag, insert base after it
        const headMatch = html.match(/<head[^>]*>/i);
        if (headMatch){
          return html.replace(/<head[^>]*>/i, headMatch[0] + '<base target="_blank">');
        }
        // if there's <html>, insert head with base
        const htmlTag = html.match(/<html[^>]*>/i);
        if (htmlTag){
          return html.replace(/<html[^>]*>/i, htmlTag[0] + '<head><base target="_blank"></head>');
        }
        // fallback: prepend base
        return '<base target="_blank">' + html;
      } catch(e){
        return '<base target="_blank">' + html;
      }
    }

    async function callGenerate(description, timeoutMs=60000){
      if (!BASE_URL || BASE_URL.includes('{{NGROK_URL')) throw new Error('Replace BASE_URL placeholder with your backend URL at the top of this file.');
      if (isGenerating) throw new Error('Generation in progress');
      isGenerating = true;
      const controller = new AbortController();
      const id = setTimeout(()=> controller.abort(), timeoutMs);
      try {
        const res = await fetch(BASE_URL + '/generate', {
          method:'POST',
          headers:{ 'Content-Type':'application/json' },
          body: JSON.stringify({ description }),
          signal: controller.signal
        });
        clearTimeout(id);
        if (!res.ok){
          const t = await res.text();
          throw new Error(`Server ${res.status}: ${t}`);
        }
        const json = await res.json();
        return json;
      } finally {
        isGenerating = false;
      }
    }

    // initial generation flow
    async function initialGenerate(description){
      if (!description || !description.trim()){ alert('Please enter a description.'); return; }
      showSplit();
      appendBubble('user', description);
      const assistant = appendBubble('assistant', 'Generating preview...');
      try {
        const data = await callGenerate(description);
        const html = data?.html || '';
        const code = data?.code || html || '';
        lastGenerated = { html, code };
        persist();
        // inject base target to keep links from navigating parent, then set iframe
        previewIframe.srcdoc = ensureBaseTarget(html || '<div style="padding:16px">⚠️ No HTML returned.</div>');
        codeEl.textContent = code || '⚠️ No code returned.';
        assistant.textContent = 'Generated preview is ready. Use the chat on the left to request edits.';
        setControlsVisible(true);
        setShowingCode(false);
      } catch(err){
        console.error(err);
        assistant.textContent = 'Generation failed: ' + (err.message || 'network error');
        previewIframe.srcdoc = `<div style="padding:16px;color:#111">⚠️ Error: ${String(err.message)}</div>`;
      }
    }

    // follow-up edit -> calls /generate with a contextual prompt
    async function followUpChange(text){
      if (!text || !text.trim()) return;
      appendBubble('user', text);
      const assistant = appendBubble('assistant', 'Applying change...');
      try {
        const combined = `${text}\n\nContext: update the previously generated site accordingly and return full HTML.`;
        const data = await callGenerate(combined);
        const html = data?.html || lastGenerated.html || '';
        const code = data?.code || html || lastGenerated.code || '';
        lastGenerated = { html, code };
        persist();
        previewIframe.srcdoc = ensureBaseTarget(html || '');
        codeEl.textContent = code || '⚠️ No code returned.';
        assistant.textContent = 'Change applied — preview updated.';
      } catch(err){
        console.error(err);
        assistant.textContent = 'Failed to apply change: ' + (err.message || 'network error');
      }
    }

    // events wiring
    heroGen.addEventListener('click', ()=> initialGenerate(heroInput.value.trim()));
    heroInput.addEventListener('keypress', (e)=> { if (e.key === 'Enter') { e.preventDefault(); heroGen.click(); }});
    heroSample.addEventListener('click', ()=> {
      const sample = 'Freelancer portfolio for Thiru Thodangi with hero, about, services, projects grid, testimonials and contact form. Modern clean UI.';
      heroInput.value = sample;
      initialGenerate(sample);
    });

    chatSend.addEventListener('click', ()=> { const t = chatInput.value.trim(); chatInput.value=''; followUpChange(t); });
    chatInput.addEventListener('keypress', (e)=> { if (e.key === 'Enter' && !e.shiftKey) { e.preventDefault(); chatSend.click(); }});
    chatClear.addEventListener('click', ()=> { chatBody.innerHTML=''; try{ sessionStorage.removeItem('invi_last'); }catch(e){} });

    toggle.addEventListener('click', ()=> setShowingCode(!showingCode));
    copyBtn.addEventListener('click', async ()=> {
      try { await navigator.clipboard.writeText(lastGenerated.code || lastGenerated.html || ''); copyBtn.textContent='Copied'; setTimeout(()=> copyBtn.textContent='Copy',1000); } catch(e){ alert('Copy failed'); }
    });
    download.addEventListener('click', ()=> {
      const content = lastGenerated.code || lastGenerated.html || '';
      const blob = new Blob([content], { type: 'text/html' });
      const url = URL.createObjectURL(blob);
      const a = document.createElement('a'); a.href=url; a.download='invi-generated.html'; document.body.appendChild(a); a.click(); a.remove(); URL.revokeObjectURL(url);
    });

    // on load try restore previous
    (function init(){
      try {
        const raw = sessionStorage.getItem('invi_last');
        if (raw){
          const parsed = JSON.parse(raw);
          if (parsed && (parsed.html || parsed.code)){
            lastGenerated = parsed;
            previewIframe.srcdoc = ensureBaseTarget(parsed.html || '');
            codeEl.textContent = parsed.code || parsed.html || '';
            // show split directly
            hero.classList.add('hidden');
            shell.classList.remove('hidden');
            setControlsVisible(true);
            setShowingCode(false);
            return;
          }
        }
      } catch(e){}
      // default informative preview while hero visible
      previewIframe.srcdoc = `<div style="font-family:Inter,Arial;padding:28px;color:#07122a">
        <h2 style="margin-top:0">INVI Generator</h2>
        <p>Enter a short description in the hero input and click Generate. After generation, the chat appears on the left and the preview is live on the right (links open in new tabs).</p>
        <p style="margin-top:12px;color:#374151">Replace BASE_URL placeholder at the top with your backend endpoint before using.</p>
      </div>`;
    })();

  })();
  </script>
</body>
</html>


Writing index.html


In [7]:
!pip install -q pyngrok


In [8]:
from pyngrok import ngrok
ngrok.kill()

In [9]:
import subprocess, time, os
from google.colab import auth
auth.authenticate_user()

# Get Gemini API key (from Colab secret)
try:
    from google.colab import userdata
    GEMINI_API = userdata.get('Gemini_API') # Using 'Gemini_API' as defined in server.js
except ImportError:
    GEMINI_API = os.environ.get("Gemini_API") # Fallback to environment variable if not in Colab

# Set environment variable for the backend server
os.environ["Gemini_API"] = GEMINI_API

# Install backend dependencies in the correct directory
!npm install express body-parser cors

# Start backend (Node server) with logging to a file
backend = subprocess.Popen(
    ["node", "server.js"],
    stdout=open("backend.log", "w"),
    stderr=subprocess.STDOUT
)

# Wait for backend to boot
time.sleep(3)

# Show last lines of backend log (to confirm server started)
!tail -n 20 backend.log


# Get ngrok auth token (from Colab secret or env var)
try:
    from google.colab import userdata
    NGROK_AUTH_TOKEN = userdata.get('invi_ngrok')
except ImportError:
    NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN")

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start backend tunnel (port 3000)
tunnel = ngrok.connect(3000, "http")
backend_ngrok_url = tunnel.public_url

print("✅ Backend available at:", backend_ngrok_url)

⠙⠹⠸⠼
up to date, audited 71 packages in 666ms
⠼
⠼16 packages are looking for funding
⠼  run `npm fund` for details
⠼
found 0 vulnerabilities
⠼✅ SuperINVI Gemini API running on port 3000
✅ Backend available at: https://389dfd8bc362.ngrok-free.app


In [10]:
with open("index.html", "r") as f:
    html = f.read()

html = html.replace("{{NGROK_URL}}", backend_ngrok_url)

with open("index.html", "w") as f:
    f.write(html)

print("✅ index.html patched with backend ngrok URL")


✅ index.html patched with backend ngrok URL


In [11]:
!nohup npx serve -l 8000 -s . > frontend.log 2>&1 &
!sleep 9
!tail -n 10 frontend.log


 INFO  Accepting connections at http://localhost:8000


In [12]:
# !pkill -f ngrok

In [13]:
frontend_tunnel = ngrok.connect(8000, "http")
print("🌍 Frontend available at:", frontend_tunnel.public_url)


🌍 Frontend available at: https://fa508c20a581.ngrok-free.app


In [14]:
!node server.js

✅ SuperINVI Gemini API running on port 3000


In [15]:
!tail -n 50 backend.log


✅ SuperINVI Gemini API running on port 3000


In [16]:
# Cell 6 - run in a code cell
!curl -s -X POST http://127.0.0.1:3000/generate -H "Content-Type: application/json" -d '{"description":"A minimal, premium e-commerce homepage for SuperINVI"}' | head -n 20


{"html":"<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n<meta charset=\"UTF-8\">\n<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">\n<title>SuperINVI</title>\n<style>\nbody {\n  font-family: sans-serif;\n  margin: 0;\n  background-color: #f4f4f4;\n  color: #333;\n}\n\nheader {\n  background-color: #fff;\n  padding: 20px;\n  text-align: center;\n  box-shadow: 0 2px 5px rgba(0,0,0,0.1);\n}\n\nheader h1 {\n  margin: 0;\n  font-size: 2em;\n}\n\n.hero {\n  background-image: url('hero-image.jpg'); /* Replace with your hero image */\n  background-size: cover;\n  background-position: center;\n  height: 500px;\n  display: flex;\n  flex-direction: column;\n  justify-content: center;\n  align-items: center;\n  text-align: center;\n  color: white;\n}\n\n.hero h2 {\n  font-size: 3em;\n  margin-bottom: 10px;\n}\n\n.hero p {\n  font-size: 1.2em;\n  margin-bottom: 20px;\n}\n\n.btn {\n  background-color: #ff5722; /* Orange color */\n  color: white;\n  padding: 15px 30px;\n  text-